In [1]:
# import os + os.chir is used to get to the llms directory to use the usermodules 
import os
os.chdir('C:/Users/AViey/OneDrive/Documents/College/UCI/Stats/stats170a/copilot/copilot/user_modules') 

from  user_modules.table_instances import Summaries, Sentiments, Base
from  user_modules.db import Session, engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

session = Session()

First we will load the sentiment and summary data

In [15]:
# summary
# Depending on your current directory, your path may be different so adjust as needed!
df_summary = pd.read_csv('../Datasets/mistral_summary_results.csv') 
df_summary.head()

,dialogue_id,Time Taken (s),GPU Usage (MB),Summary
0,1,0.554711,213.601387,Doctor Hawkins greets Mr. Smith and asks why h...
1,2,0.554711,213.601387,Person1 greets Person2 and checks Ricky's vacc...
2,3,0.554711,213.601387,Person1 lost a set of five keys with a small f...
3,4,0.554711,213.601387,Person1 was upset that Person2 had a girlfrien...
4,5,0.554711,213.601387,Malik approaches Wen and Nikki at the dance fl...


In [16]:
# sentiment
df_sentiment = pd.read_csv('../Datasets/mistral_evals.csv')
df_sentiment.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment,generated_sentiment,load_before,load_after,mem_before,mem_after,time_taken
0,1,training,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",positive,0.0,0.00,0.31,20247.0,20247.0,0.121527
1,2,training,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,positive,0.0,0.31,0.31,20247.0,20247.0,0.137223
2,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive,1.0,0.45,0.45,20247.0,20247.0,0.134461
3,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative,-1.0,0.45,0.58,20247.0,20247.0,0.120814
4,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive,1.0,0.58,0.47,20247.0,20247.0,0.106811


Merge the dataframes together with only the information to upload (exclude the gpu, memory, time take for now). 

In [23]:
merged_df = pd.merge(df_summary[['dialogue_id', 'Summary']], 
                  df_sentiment[['dialogue_id', 'actual_summary', 'actual_sentiment', 'generated_sentiment']],
                  on='dialogue_id',
                  how='inner')

merged_df.head()


,dialogue_id,Summary,actual_summary,actual_sentiment,generated_sentiment
0,1,Doctor Hawkins greets Mr. Smith and asks why h...,"Mr. Smith's getting a check-up, and Doctor Haw...",positive,0.0
1,2,Person1 greets Person2 and checks Ricky's vacc...,Mrs Parker takes Ricky for his vaccines. Dr. P...,positive,0.0
2,3,Person1 lost a set of five keys with a small f...,#Person1#'s looking for a set of keys and asks...,positive,1.0
3,4,Person1 was upset that Person2 had a girlfrien...,#Person1#'s angry because #Person2# didn't tel...,negative,-1.0
4,5,Malik approaches Wen and Nikki at the dance fl...,Malik invites Nikki to dance. Nikki agrees if ...,positive,1.0


Convert generated sentiment to that of the same format and dtype of the actual sentiment.

In [24]:
merged_df.dtypes

dialogue_id              int64
Summary                 object
actual_summary          object
actual_sentiment        object
generated_sentiment    float64
dtype: object

In [25]:
"""
converts sentiment value to its corresponding label
"""
def sentiment_format(sentiment_value: float) -> str:
    sentiment = {-1:'negative', 0:'neutral', 1: 'positive'}
    return sentiment[int(sentiment_value)]

In [26]:
merged_df["generated_sentiment"] = merged_df["generated_sentiment"].apply(sentiment_format)
merged_df.head()

,dialogue_id,Summary,actual_summary,actual_sentiment,generated_sentiment
0,1,Doctor Hawkins greets Mr. Smith and asks why h...,"Mr. Smith's getting a check-up, and Doctor Haw...",positive,neutral
1,2,Person1 greets Person2 and checks Ricky's vacc...,Mrs Parker takes Ricky for his vaccines. Dr. P...,positive,neutral
2,3,Person1 lost a set of five keys with a small f...,#Person1#'s looking for a set of keys and asks...,positive,positive
3,4,Person1 was upset that Person2 had a girlfrien...,#Person1#'s angry because #Person2# didn't tel...,negative,negative
4,5,Malik approaches Wen and Nikki at the dance fl...,Malik invites Nikki to dance. Nikki agrees if ...,positive,positive


In [27]:
merged_df.rename(columns={'Summary': 'generated_summary'}, inplace=True)
merged_df.columns

Index(['dialogue_id', 'generated_summary', 'actual_summary',
       'actual_sentiment', 'generated_sentiment'],
      dtype='object')

In [60]:
merged_df.head(1)

,dialogue_id,generated_summary,actual_summary,actual_sentiment,generated_sentiment
0,1,Doctor Hawkins greets Mr. Smith and asks why h...,"Mr. Smith's getting a check-up, and Doctor Haw...",positive,neutral


Now we can generate the scores: rouge, meteor, bert.

In [45]:
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from bert_score import score
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AViey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AViey\AppData\Roaming\nltk_data...


In [68]:
def compute_summary_scores(row: pd.Series) -> pd.Series:

    # gather the text from the current row
    actual_summary: str = row['actual_summary']
    generated_summary: str = row['generated_summary']

    # generate the rouge score and add to the generated corresponding column
    rouge = Rouge()
    r_scores = rouge.get_scores(generated_summary, actual_summary, avg = True)
    row['rouge_1'] = r_scores['rouge-1']['f']
    row['rouge_2'] = r_scores['rouge-2']['f']
    row['rouge_l'] = r_scores['rouge-l']['f']

    # generate the bert score and add to the generated corresponding column
    bert_p, bert_r, bert_f = score([generated_summary], [actual_summary], lang='en', verbose=False)
    row['bert_precision'] = bert_p.mean().item()
    row['bert_recall'] = bert_r.mean().item()
    row['bert_f1'] = bert_f.mean().item()

    # meteor requires the both generated and actual summaries to be tokenized 
    # before they can be evaluated
    tokenized_actual_summary = word_tokenize(actual_summary)
    tokenized_generated_summary = word_tokenize(generated_summary)

    # generate the meteor score and add to the generated corresponding column
    meteor = meteor_score([tokenized_actual_summary], tokenized_generated_summary)
    row['meteor'] = meteor

    return row
    

In [69]:
merged_df = merged_df.apply(compute_summary_scores, axis = 1)
merged_df.head()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

,dialogue_id,generated_summary,actual_summary,actual_sentiment,generated_sentiment,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor
0,1,Doctor Hawkins greets Mr. Smith and asks why h...,"Mr. Smith's getting a check-up, and Doctor Haw...",positive,neutral,0.351351,0.159091,0.324324,0.888796,0.906006,0.897319,0.505513
1,2,Person1 greets Person2 and checks Ricky's vacc...,Mrs Parker takes Ricky for his vaccines. Dr. P...,positive,neutral,0.250000,0.000000,0.218750,0.836484,0.907462,0.870528,0.335816
2,3,Person1 lost a set of five keys with a small f...,#Person1#'s looking for a set of keys and asks...,positive,positive,0.367347,0.074074,0.285714,0.880228,0.880754,0.880491,0.363356
3,4,Person1 was upset that Person2 had a girlfrien...,#Person1#'s angry because #Person2# didn't tel...,negative,negative,0.233333,0.105263,0.166667,0.868038,0.871339,0.869685,0.304503
4,5,Malik approaches Wen and Nikki at the dance fl...,Malik invites Nikki to dance. Nikki agrees if ...,positive,positive,0.392857,0.120482,0.392857,0.869606,0.924768,0.896339,0.529660


In [124]:
merged_df["model_id"] = "mistral 7b"
merged_df.head()

,dialogue_id,generated_summary,actual_summary,actual_sentiment,generated_sentiment,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor,model_id
0,1,Doctor Hawkins greets Mr. Smith and asks why h...,"Mr. Smith's getting a check-up, and Doctor Haw...",positive,neutral,0.351351,0.159091,0.324324,0.888796,0.906006,0.897319,0.505513,mistral 7b
1,2,Person1 greets Person2 and checks Ricky's vacc...,Mrs Parker takes Ricky for his vaccines. Dr. P...,positive,neutral,0.250000,0.000000,0.218750,0.836484,0.907462,0.870528,0.335816,mistral 7b
2,3,Person1 lost a set of five keys with a small f...,#Person1#'s looking for a set of keys and asks...,positive,positive,0.367347,0.074074,0.285714,0.880228,0.880754,0.880491,0.363356,mistral 7b
3,4,Person1 was upset that Person2 had a girlfrien...,#Person1#'s angry because #Person2# didn't tel...,negative,negative,0.233333,0.105263,0.166667,0.868038,0.871339,0.869685,0.304503,mistral 7b
4,5,Malik approaches Wen and Nikki at the dance fl...,Malik invites Nikki to dance. Nikki agrees if ...,positive,positive,0.392857,0.120482,0.392857,0.869606,0.924768,0.896339,0.529660,mistral 7b


Upload the data to the postgre SQL server:

1. Upload summaries data
2. upload sentiment data

Upload summaries data

In [2]:
merged_df = pd.read_csv("C:/Users/AViey/OneDrive/Documents/College/UCI/Stats/stats170a/copilot/mistral.csv")
merged_df.head()

,dialogue_id,generated_summary,actual_summary,actual_sentiment,generated_sentiment,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor,model_id
0,1,Doctor Hawkins greets Mr. Smith and asks why h...,"Mr. Smith's getting a check-up, and Doctor Haw...",positive,neutral,0.351351,0.159091,0.324324,0.888796,0.906006,0.897319,0.505513,mistral 7b
1,2,Person1 greets Person2 and checks Ricky's vacc...,Mrs Parker takes Ricky for his vaccines. Dr. P...,positive,neutral,0.250000,0.000000,0.218750,0.836484,0.907462,0.870528,0.335816,mistral 7b
2,3,Person1 lost a set of five keys with a small f...,#Person1#'s looking for a set of keys and asks...,positive,positive,0.367347,0.074074,0.285714,0.880228,0.880754,0.880491,0.363356,mistral 7b
3,4,Person1 was upset that Person2 had a girlfrien...,#Person1#'s angry because #Person2# didn't tel...,negative,negative,0.233333,0.105263,0.166667,0.868038,0.871339,0.869685,0.304503,mistral 7b
4,5,Malik approaches Wen and Nikki at the dance fl...,Malik invites Nikki to dance. Nikki agrees if ...,positive,positive,0.392857,0.120482,0.392857,0.869606,0.924768,0.896339,0.529660,mistral 7b


In [3]:
summaries = merged_df[["dialogue_id", "model_id", "generated_summary", "rouge_1", "rouge_2", "rouge_l", "bert_precision", "bert_recall", "bert_f1", "meteor"]]
summaries.head()

,dialogue_id,model_id,generated_summary,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor
0,1,mistral 7b,Doctor Hawkins greets Mr. Smith and asks why h...,0.351351,0.159091,0.324324,0.888796,0.906006,0.897319,0.505513
1,2,mistral 7b,Person1 greets Person2 and checks Ricky's vacc...,0.250000,0.000000,0.218750,0.836484,0.907462,0.870528,0.335816
2,3,mistral 7b,Person1 lost a set of five keys with a small f...,0.367347,0.074074,0.285714,0.880228,0.880754,0.880491,0.363356
3,4,mistral 7b,Person1 was upset that Person2 had a girlfrien...,0.233333,0.105263,0.166667,0.868038,0.871339,0.869685,0.304503
4,5,mistral 7b,Malik approaches Wen and Nikki at the dance fl...,0.392857,0.120482,0.392857,0.869606,0.924768,0.896339,0.529660


In [4]:
# convert for batch form
dict_converted_summaries = summaries.to_dict(orient='records')
print(dict_converted_summaries[:1])

[{'dialogue_id': 1, 'model_id': 'mistral 7b', 'generated_summary': "Doctor Hawkins greets Mr. Smith and asks why he's there for a check-up, which he hasn't had in 5 years. Mr. Smith mentions that he only goes when something is wrong. Doctor Hawkins explains the importance of early detection and encourages him to come annually. Mr. Smith admits to smoking and Doctor Hawkins advises him of the health risks and offers help to quit.", 'rouge_1': 0.3513513467165814, 'rouge_2': 0.1590909046720042, 'rouge_l': 0.3243243196895544, 'bert_precision': 0.8887962102890015, 'bert_recall': 0.9060062170028688, 'bert_f1': 0.8973186612129211, 'meteor': 0.5055129533678757}]


In [5]:
# upload summaries 
session.bulk_insert_mappings(Summaries, dict_converted_summaries)
session.commit()

2024-05-03 19:25:25,769 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-05-03 19:25:25,770 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-03 19:25:25,953 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-03 19:25:25,953 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-03 19:25:26,190 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-03 19:25:26,190 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-03 19:25:26,407 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-03 19:25:26,513 INFO sqlalchemy.engine.Engine INSERT INTO summaries (dialogue_id, model_id, generated_summary, rouge_1, rouge_2, rouge_l, bert_precision, bert_recall, bert_f1, meteor) VALUES (%(dialogue_id__0)s, %(model_id__0)s, %(generated_summary__0)s, %(rouge_1__0)s, %(rouge_2__0)s, %(rouge_l ... 205693 characters truncated ...  %(rouge_l__999)s, %(bert_precision__999)s, %(bert_recall__999)s, %(bert_f1__999)s, %(meteor__999)s)
2024-05-03 19:25:26,522 INFO sqlalchemy

Upload Sentiment Data

In [3]:
sentiments = merged_df[['dialogue_id', 'model_id', 'generated_sentiment']]
sentiments.head()

,dialogue_id,model_id,generated_sentiment
0,1,mistral 7b,neutral
1,2,mistral 7b,neutral
2,3,mistral 7b,positive
3,4,mistral 7b,negative
4,5,mistral 7b,positive


In [4]:
dict_converted_sentiments = sentiments.to_dict(orient='records')
print(dict_converted_sentiments[:1])

[{'dialogue_id': 1, 'model_id': 'mistral 7b', 'generated_sentiment': 'neutral'}]


In [5]:
session.bulk_insert_mappings(Sentiments, dict_converted_sentiments)
session.commit()

2024-05-03 19:41:05,205 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-05-03 19:41:05,206 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-03 19:41:05,441 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-03 19:41:05,442 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-03 19:41:05,670 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-03 19:41:05,671 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-03 19:41:05,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-03 19:41:06,000 INFO sqlalchemy.engine.Engine INSERT INTO sentiments (dialogue_id, model_id, generated_sentiment) VALUES (%(dialogue_id__0)s, %(model_id__0)s, %(generated_sentiment__0)s), (%(dialogue_id__1)s, %(model_id__1)s, %(generated_sentiment__1)s), (%(dialogue_id__2)s, %(model_id__2)s, %(g ... 72393 characters truncated ... generated_sentiment__998)s), (%(dialogue_id__999)s, %(model_id__999)s, %(generated_sentiment__999)s)
2024-05-03 19:41:06,000 INFO sqlalchemy.

In [7]:
session.close()